In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter1d as g1d
from scipy.stats import norm
import matplotlib.pylab as plt
from scipy.interpolate import interp1d
from matplotlib import cm


# Calibration invariance of LUT2, rsp, efficiency

$Observation$ (e.g. `ISGRI` spectral histograms) can be represented as transformation of some true $Model$ with $Response$:

$$
Observation~=~ Response \times Model
$$

Note that $Observation$ does not belong to the same space as $Model$. For example, when $Model$ is vectors of flux in incident photon energy bins, $Observation$ is detected 
counts in detector channels. 

For `ISGRI`, each pixel detector channels are biparametric, described by $\mathrm{ISGRI\_PHA}$ and $\mathrm{ISGRI\_RT}$ integers.

$$
CountSpectrum\left[\mathrm{ISGRI\_PHA,~ISGRI\_RT}\right] = \sum_{Energy}~Response\left[\mathrm{ISGRI\_PHA,~ISGRI\_RT},~\mathrm{Energy}\right] \times ModelFlux\left[\mathrm{Energy}\right]
$$

Where $CountSpectrum$ is as directly returned by the pixels. This equation is valid for a single pixel at single moment of time. All pixels differ, and also evolve (in long term as well as in short disruptions of several different kinds). So it is almost always necessary to conside that (assuming that $ModelFlux$ is constant in time):

$$
CountSpectrum\left[\mathrm{ISGRI\_PHA,~ISGRI\_RT,~ISGRI\_PIXEL,~Time}\right] = \sum_{Energy}~Response\left[\mathrm{ISGRI\_PHA,~ISGRI\_RT,~ISGRI\_PIXEL,~Time},~\mathrm{Energy}\right] \times ModelFlux\left[\mathrm{Energy}\right]
$$

It means that `ISGRI` spectra are 



This way, the entire transformation from the incident flux to observation is contained in the response.



$$
CountSpectrum\left[\mathrm{ISGRI\_ENERGY}\right] = Response\left[\mathrm{ISGRI\_ENERGY},~\mathrm{Energy}\right] \times ModelFlux\left[\mathrm{Energy}\right]
$$


## ISGRI Efficiency components

### Pixel efficiency

### NOMEX absorption

### Uniformity

In [ ]:
# TODO: measure or mimick gain from HE line alone, and from LE line alone. find which one mismatches

# TODO:
# * reconstruct with det
# * reconstruct with det and morph

# TODO: 
# reconstruct effi
# reconstruct lut2
# reconstruct rsp

